In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tvb.simulator.lab as tsl
import mne_connectivity 
import tvb
from scipy import signal
from importlib import reload
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn.cluster import KMeans
import scipy 
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd
import netplotbrain

import functions as fn

In [ ]:
def DFC_visu_pipeline(data, cor_method, num_clusters=3, window_size=2000, overlap=1000,tmax=30, intrahemispheric=False, square=[0,0],animated_visu=True,ts=0.5):
    '''
    This function performs visualisations regarding Dynamical Functional connectivity (DFC). 

    ------------
    INPUTS
    ------------

    data : array_like, shape = (signal_duration/timestep,num_regions) 
        The signal data, typically eeg signal or source level signal. 
    cor_method : string
        The correlation method to build the connectivity matrices. Can be chosen over 'Pearson', 'Spearman', 'Coherence', 'Phase-lock' and 'wpli'.
    num_clusters : int, default = 3
        Number of clusters for the k-means clustering.
    window_size : int, default = 2000  
        window_duration[ms]/timestep[ms]
    overlap : int or float, default = 1000 
        Time of overlap [ms] between two succint windows. 
    tmax : int, default = 30
        Time of the simulation, for visualisation purpose. 
    intrahemispheric : bool, default = False
        Visualisation of clustering of a single square of the connectivity matrix. 
    square : 2x2 list, default = [0,0]
        If intrahemispheric = True, specifies the square of the matrix to perform the analysis on. 
    animated_visu : bool, default = True
        Plot and save an animated plot showing the evolution of correlation matrices and pca visualisation. 


    ------------
    OUTPUTS
    ------------
    closest_mats : array_like, shape = (num_clusters, num_regions,num_regions)
        Closest connectivity matrix to every cluster. 
    most_repeating_states : array_like, shape = (num_clusters, num_regions,num_regions)
         Kmeans clusters centroids. 
    correlation_matrices : array_like, shape = (num_windows, num_regions,num_regions)
        Set of correlation matrics, one per window. 
    cluster_labels : list, len = num_clusters
        Cluster labels.
    distances : array_like, shape = (num_clusters, num_windows)
         Matrix storing the distances between each connectivity matrix and each cluster. 
    pca : Sklearn PCA object.
        PCA object used for the connectivity matrices dimension reduction.   
    reduced_data : array_like
        3D PCA reduced data.
    '''
    closest_mats,most_repeating_states, correlation_matrices, cluster_labels,num_clusters = fn.build_FCmat(data,cor_method,num_clusters,window_size,overlap,tmax,intrahemispheric,square=square)
    distances = fn.compute_clst_dist(most_repeating_states, correlation_matrices, cluster_labels,tmax,num_clusters) 
    pca, reduced_data = fn.plot_PCA(correlation_matrices, most_repeating_states, cluster_labels,num_clusters)
    if animated_visu : 
        fn.animated_plot(correlation_matrices,reduced_data,cluster_labels)

    return closest_mats,most_repeating_states, correlation_matrices, cluster_labels, distances, pca, reduced_data
    


# Define paths

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'

# Surface based

## Before stim

### Stroke

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [5]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats5,most_repeating_states5, correlation_matrices5, cluster_labels5, distances5, pca5, reduced_data5 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
fig, axs = plt.subplots(1, num_clusters, figsize=(15, 5))
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Position of the colorbar

for a in range(len(closest_mats5)):
    im = axs[a].imshow(closest_mats5[a], cmap='viridis')
    axs[a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[a].axhline(y=34, color='white', linewidth=2)
    axs[a].axhline(y=49, color='white', linewidth=2)
    axs[a].axvline(x=34, color='white', linewidth=2)
    axs[a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')

fig.colorbar(im, cax=cbar_ax)
fig.suptitle(f'Closest matrix to the cluster centers, Stroke', y=0.92)
plt.show()


### Healthy

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats7,most_repeating_states7, correlation_matrices7, cluster_labels7,distances7, pca7, reduced_data7 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [10]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=30000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats10,most_repeating_states10, correlation_matrices10, cluster_labels10,distances10, pca10, reduced_data10 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## matrices figure

In [ ]:
fig, axs = plt.subplots(2, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats5)):
    im = axs[0, a].imshow(closest_mats5[a], cmap='viridis')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)

    
    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[0, 1].set_title('Stroke',fontsize=22)  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats7)):
    im = axs[1, a].imshow(closest_mats7[a], cmap='viridis')
    axs[1, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[1, a].axhline(y=34, color='white', linewidth=2)
    axs[1, a].axhline(y=49, color='white', linewidth=2)
    axs[1, a].axvline(x=34, color='white', linewidth=2)
    axs[1, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[1, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[1, 1].set_title('Healthy', fontsize=22)  # Add title for axs[1]

# Place colorbar at the bottom horizontally
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])

fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
plt.show()




## 2d PCA

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(15,5))

ext_data=np.concatenate((correlation_matrices5,most_repeating_states5))
data=np.array(ext_data).reshape(len(correlation_matrices5)+num_clusters,len(correlation_matrices5[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[0].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels5, palette='Set1', ax=axs[0])
#axs[0].legend(loc='upper left')

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[0]
)

axs[0].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[0].set_title(
    "Stroke", fontsize=22
)
axs[0].set_xlim(x_min-7, x_max)
axs[0].set_xticks(())
axs[0].set_yticks(())

ext_data=np.concatenate((correlation_matrices7,most_repeating_states7))
data=np.array(ext_data).reshape(len(correlation_matrices7)+num_clusters,len(correlation_matrices7[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[1].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels7, palette='Set1', ax=axs[1])

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[1]
)

axs[1].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[1].set_title(
    "Healthy", fontsize=22
)
axs[1].set_xlim(x_min-7, x_max)
axs[1].set_xticks(())
axs[1].set_yticks(())
axs[1].get_legend().remove()

## Proximity plots

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20, 15))
time = np.linspace(1,30,len(correlation_matrices5))

colors=['red','blue','green','orange','black','purple']


for a in range(num_clusters):    
    axs[0].plot(time, distances5[a, :], label=f'State {str(a)}', c=colors[a])
    axs[1].plot(time, distances7[a, :], label=f'State {str(a)}', c=colors[a])


axs[0].set_title('Stroke', fontsize=30)
axs[0].set_ylabel('1-distance',  fontsize=24)
axs[0].legend(fontsize=20)
axs[0].tick_params(axis='y', labelsize=18)
axs[0].tick_params(axis='x', labelsize=0)
axs[0].set_ylim(0.6,0.75)

axs[1].set_title('Healthy', fontsize=30)
axs[1].set_xlabel('Time [s]', fontsize=22)
axs[1].set_ylabel('1-distance',  fontsize=22)
axs[1].set_ylim(0.6,0.75)

axs[1].tick_params(axis='x', labelsize=18)
axs[1].tick_params(axis='y', labelsize=18)

plt.show()


In [ ]:
fig, axs = plt.subplots(4, num_clusters, figsize=(25, 15))
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Position of the colorbar
time = np.linspace(1,30,len(correlation_matrices))
colors=['red','blue','green','orange','black','purple']

# Plot closest_mats5
for a in range(len(closest_mats5)):
    im = axs[0, a].imshow(closest_mats5[a], cmap='plasma')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')
axs[0, 1].set_title('Stroke')  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats7)):
    im = axs[2, a].imshow(closest_mats7[a], cmap='plasma')
    axs[2, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[2, a].axhline(y=34, color='white', linewidth=2)
    axs[2, a].axhline(y=49, color='white', linewidth=2)
    axs[2, a].axvline(x=34, color='white', linewidth=2)
    axs[2, a].axvline(x=49, color='white', linewidth=2)

    if a == 0:
        axs[2, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical')
        axs[2, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical')
        axs[2, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical')
axs[2, 1].set_title('Healthy')  # Add title for axs[1]

for a in range(num_clusters):
    axs[1, 1].plot(time, distances5[a, :], label=str(a), c=colors[a])
    axs[3, 1].plot(time, distances7[a, :], label=str(a), c=colors[a])

# Set the limits of the y-axis for the distance plots
axs[1, 1].set_ylim([0, 1])
axs[3, 1].set_ylim([0, 1])

# Remove unnecessary subplots
axs[1, 0].remove()
axs[3, 0].remove()
axs[1, 2].remove()
axs[3, 2].remove()

# Set common x and y labels for the remaining subplots
axs[1, 1].set_xlabel('Time [s]')
axs[1, 1].set_ylabel('1 - Relative distance')
axs[3, 1].set_xlabel('Time [s]')
axs[3, 1].set_ylabel('1 - Relative distance')

# Add legends to the remaining subplots
axs[1, 1].legend()
axs[3, 1].legend()

fig.colorbar(im, cax=cbar_ax)
fig.suptitle(f'Comparison of network states without stimulation between stroke and healthy patient', y=0.92)
plt.show()


## During stim

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [5,10]:#[2,5,7,8,9,10,11,12]    
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=30000
    t2=60000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca12, reduced_data = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Figure poster

In [ ]:
path_conn = f'D:/Fariba/15Hz/'
for i in [7]:#[2,5,7,8,9,10,11,12]    
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment11.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=30000
    t2=60000

    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats_stim,most_repeating_states_stim, correlation_matrices_stim, cluster_labels_stim,distances_stim, pca12_stim, reduced_data_stim = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
fig, axs = plt.subplots(2, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats7)):
    im = axs[0, a].imshow(closest_mats7[a], cmap='viridis')
    axs[0, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[0, a].axhline(y=34, color='white', linewidth=2)
    axs[0, a].axhline(y=49, color='white', linewidth=2)
    axs[0, a].axvline(x=34, color='white', linewidth=2)
    axs[0, a].axvline(x=49, color='white', linewidth=2)
    
    # Outline the 13th row and 55th column
    axs[0, a].axhline(y=12, color='red', linewidth=0.5)
    axs[0, a].axvline(x=54, color='red', linewidth=0.5)
    axs[0, a].axhline(y=14, color='red', linewidth=0.5)
    axs[0, a].axvline(x=56, color='red', linewidth=0.5)

    if a == 0:
        axs[0, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[0, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[0, 1].set_title('Before stimulation',fontsize=14)  # Add title for axs[0]

# Plot closest_mats7
for a in range(len(closest_mats_stim)):
    im = axs[1, a].imshow(closest_mats_stim[a], cmap='viridis')
    axs[1, a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[1, a].axhline(y=34, color='white', linewidth=2)
    axs[1, a].axhline(y=49, color='white', linewidth=2)
    axs[1, a].axvline(x=34, color='white', linewidth=2)
    axs[1, a].axvline(x=49, color='white', linewidth=2)
    
    # Outline the 13th row and 54th column
    axs[1, a].axhline(y=12, color='red', linewidth=0.5)
    axs[1, a].axvline(x=54, color='red', linewidth=0.5)
    axs[1, a].axhline(y=14, color='red', linewidth=0.5)
    axs[1, a].axvline(x=56, color='red', linewidth=0.5)

    if a == 0:
        axs[1, a].text(-2.8, 25, 'Left Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 50, 'Subcortical', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[1, a].text(-2.8, 78, 'Right Cortical', color='black', ha='center', rotation='vertical',fontsize=11)
axs[1, 1].set_title('During stimulation', fontsize=14)  # Add title for axs[1]

# Place colorbar at the bottom horizontally
cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])

fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
plt.show()


In [ ]:
fig,axs=plt.subplots(1,2,figsize=(15,5))

ext_data=np.concatenate((correlation_matrices_stim,most_repeating_states_stim))
data=np.array(ext_data).reshape(len(correlation_matrices_stim)+num_clusters,len(correlation_matrices_stim[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[0].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels_stim, palette='Set1', ax=axs[0])
#axs[0].legend(loc='upper left')

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[0]
)

axs[0].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[0].set_title(
    "Stroke", fontsize=22
)
axs[0].set_xlim(x_min-7, x_max)
axs[0].set_xticks(())
axs[0].set_yticks(())

ext_data=np.concatenate((correlation_matrices7,most_repeating_states7))
data=np.array(ext_data).reshape(len(correlation_matrices7)+num_clusters,len(correlation_matrices7[0])**2)
reduced_data = PCA(n_components=2).fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


axs[1].plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1], linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10, hue=cluster_labels7, palette='Set1', ax=axs[1])

# Plot the centroids as a white X
centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0, num_clusters-1, num_clusters),
    palette='Set1',
    legend=False,
    ax=axs[1]
)

axs[1].quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid", width=0.001, headwidth=20)

axs[1].set_title(
    "Healthy", fontsize=22
)
axs[1].set_xlim(x_min-7, x_max)
axs[1].set_xticks(())
axs[1].set_yticks(())
axs[1].get_legend().remove()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(25, 20))
time = np.linspace(1,30,len(correlation_matrices))

colors=['red','blue','green','orange','black','purple']


for a in range(num_clusters):    
    axs[0].plot(time, distances7[a, :], label=f'State {str(a)}', c=colors[a])
    axs[1].plot(time, distances_stim[a, 4:], label=f'State {str(a)}', c=colors[a])


axs[0].set_title('Before stimulation', fontsize=30)
axs[0].set_xlabel('Time [s]', fontsize=24)
axs[0].set_ylabel('1-Distance',  fontsize=24)
axs[0].legend()

axs[1].set_title('After stimulation', fontsize=30)
axs[1].set_xlabel('Time [s]', fontsize=24)
axs[1].set_ylabel('1-Distance',  fontsize=24)
axs[1].legend()

fig.suptitle(f'Proximity to each state over time', y=0.94,fontsize=35)

plt.show()




## Results template

In [ ]:
path_conn = 'D:/Fariba/SC'
path_save = f'D:/Timing_optimisation'
G=[7]
#G=[1,1.5,2,3,4,5,6,7,8,9,10,12]
for g in range(len(G)):
    path_conn = 'D:/Fariba/SC'
    path_save = f'D:/Timing_optimisation'
    data = np.load(f'{path_save}/20minsim_dt_0.5_G_{7}_sigma_1e-07.npz')
    data_time=data['traw_time']
    delta_t = data_time[1]-data_time[0]
    sf = 1/(delta_t*1e-3)
    win = 1*sf

    t1 = 1000
    t2 = 2400000 #unit second
    time_corr_id= np.where((data_time>=t1)*(data_time<=t2))
    idc1 = time_corr_id[0][0]
    idc2= time_corr_id[0][-1]
    time_corr=data_time[idc1:idc2]
    data_corr=data['traw_y1y2'][idc1:idc2]
    print(np.array(data_corr).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats_temp,most_repeating_states_temp, correlation_matrices_temp, cluster_labels_temp,distances_temp, pca12_temp, reduced_data_temp = DFC_visu_pipeline(data_corr,meth,animated_visu=False, window_size=10000,overlap=5000,tmax=t2/1000, intrahemispheric=False)

In [ ]:
components = pca12_temp.components_
    for comp in components : 
        plt.figure()
        plt.imshow(comp.reshape(int(np.sqrt(len(comp))),int(np.sqrt(len(comp)))),cmap='bwr')
        plt.colorbar();plt.clim([-0.05,0.05])

In [ ]:
plot_PCs(pca12_temp)

In [ ]:
fig, axs = plt.subplots(1, num_clusters, figsize=(15, 10))

# Plot closest_mats5
for a in range(len(closest_mats_temp)):
    im = axs[ a].imshow(closest_mats_temp[a], cmap='viridis')
    axs[ a].axis('off')  # Remove axis labels for cleaner visualization
    
    # Add lines between lines and columns 34 and 49
    axs[ a].axhline(y=37, color='white', linewidth=2)
    axs[ a].axvline(x=37, color='white', linewidth=2)

    
    if a == 0:
        axs[ a].text(-2.8, 33, 'Right hemisphere', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[ a].text(-2.8, 70, 'Left hemisphere', color='black', ha='center', rotation='vertical',fontsize=11)
        axs[ a].text(18, 80, 'Right hemisphere', color='black', ha='center', rotation='horizontal',fontsize=11)
        axs[ a].text(55, 80, 'Left hemisphere', color='black', ha='center', rotation='horizontal',fontsize=11)

# Place colorbar on the right side vertically
cbar_ax = fig.add_axes([0.92, 0.325, 0.02, 0.34])

fig.colorbar(im, cax=cbar_ax, orientation='vertical')
plt.show()



In [ ]:
ext_data=np.concatenate((correlation_matrices_temp,most_repeating_states_temp))
data=np.array(ext_data).reshape(len(correlation_matrices_temp)+num_clusters,len(correlation_matrices_temp[0])**2)
pca_2d_temp=PCA(n_components=2)
reduced_data = pca_2d_temp.fit_transform(data)


x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1



u=np.diff(reduced_data[:-num_clusters, 0])
v=np.diff(reduced_data[:-num_clusters, 1])
pos_x = reduced_data[:-num_clusters-1, 0] + u/2
pos_y = reduced_data[:-num_clusters-1, 1] + v/2
norm = np.sqrt(u**2+v**2) 


plt.plot(reduced_data[:-num_clusters, 0], reduced_data[:-num_clusters, 1],linewidth=0.5)
sns.scatterplot(x=reduced_data[:-num_clusters, 0], y=reduced_data[:-num_clusters, 1], size=10)#,hue=cluster_labels,palette='Set1')



plt.quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid",width=0.001,headwidth=20)

# Plot the centroids as a white X
'''centroids = reduced_data[-num_clusters:]
sns.scatterplot(
    x=centroids[:, 0],
    y=centroids[:, 1],
    marker="x",
    s=600,
    hue=np.linspace(0,num_clusters-1,num_clusters),
    palette='Set1',
    legend=False
)'''


plt.xlim(x_min-7, x_max)
plt.xticks(())
plt.yticks(())
plt.legend('',frameon=False)
plt.show()

In [ ]:
coords=np.array([[-13.5,0.4],
        [35,0.4],
        [7,-6],
        [7,7]])
for coord in coords : 
    plt.figure()
    plt.imshow(pca_2d_temp.inverse_transform(coord).reshape(76,76),cmap='viridis')
    plt.axis('off')


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 7.5))

components = pca_2d_temp.components_
for c, comp in enumerate(components):
    im=axs[c].imshow(comp.reshape(int(np.sqrt(len(comp))), int(np.sqrt(len(comp)))), cmap='bwr')
    axs[c].axis('off')  # Remove axis labels for cleaner visualization
    axs[c].set_title(f'PC {c+1}')


# Place colorbar on the right side vertically
cbar_ax = fig.add_axes([0.92, 0.255, 0.02, 0.47])

fig.colorbar(im, cax=cbar_ax, orientation='vertical', cmap='bwr')
plt.show()

In [ ]:

time = np.linspace(1, 2400, len(correlation_matrices_temp))

colors = ['red', 'blue', 'green', 'orange', 'black', 'purple']

plt.figure(figsize=(16, 8))  # Increase the figure size

for a in range(num_clusters):
    plt.plot(time, distances_temp[a, :], label=f'State {str(a)}', c=colors[a])

plt.xlabel('Time [s]', fontsize=24)
plt.ylabel('1- relative distance', fontsize=24)
plt.legend(fontsize=20)
plt.tick_params(axis='y', labelsize=18)
plt.tick_params(axis='x', labelsize=18)
plt.show()

## Results dif noise, G

### TVB2

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [2]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats2,most_repeating_states2, correlation_matrices2, cluster_labels2,distances2, pca2, reduced_data2 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB5

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [5]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats5,most_repeating_states5, correlation_matrices5, cluster_labels5,distances5, pca5, reduced_data5 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

In [ ]:
plot_PCs(pca5)

### TVB7

In [ ]:
path_conn = f'D:/Fariba/15Hz_2/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment55.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats7_2,most_repeating_states7_2, correlation_matrices7_2, cluster_labels7_2,distances7_2, pca7_2, reduced_data7_2 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=950,tmax=t2/1000, intrahemispheric=False)

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [7]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods :
        closest_mats7,most_repeating_states7, correlation_matrices7, cluster_labels7,distances7, pca7, reduced_data7 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB8

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [8]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats8,most_repeating_states8, correlation_matrices8, cluster_labels8,distances8, pca8, reduced_data8 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

### TVB9

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
for i in [9]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats9,most_repeating_states9, correlation_matrices9, cluster_labels9,distances9, pca9, reduced_data9 = DFC_visu_pipeline(source_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)

## Intrahemispheric eeg

In [ ]:
file_path=f'D:/Fariba/5min_surface/'
Subjects=['TVB2','TVB5','TVB7','TVB8','TVB9']#,'TVB10','TVB11','TVB12']
right_indices=[]
left_indices=[]
middle_indices=[]
ordered_ind=[]
for i, subj in enumerate (Subjects) : 
    conn_file=file_path+subj+'/eeg_1010.txt'
    r_ind=[]
    l_ind=[]
    m_ind=[]
    with open(conn_file) as f:
        lines = f.readlines()
    for j, line in enumerate(lines) : 
        words = line.split()
        # Get the first word (element before the first space)
        first_word = words[0]
        print(first_word)
        if first_word[-1].isdigit() : 
            last=int(first_word[-1])
            if last%2==0 : 
                r_ind.append(j)
                print('right')
            elif last%2==1 : 
                l_ind.append(j)
                print('left')
        else :
                m_ind.append(j)
                print('middle')    

    right_indices.append(r_ind)
    left_indices.append(l_ind)
    middle_indices.append(m_ind)
    ordered_ind.append(np.concatenate((l_ind,m_ind,r_ind)))

In [ ]:
ordered_ind=np.concatenate((left_indices[1],middle_indices[1],right_indices[1]))

In [ ]:
ordered_ind

### TVB2

## all subjects eeg

In [ ]:
path_conn = f'D:/Fariba/5min_surface/'
closest_mats_joined=[]
for i in [2,5,7,8,9]:#[2,5,7,8,9,10,11,12]:
    subj = 'TVB'+str(i)
    path_data_eeg =  f'{path_conn}/{subj}/'
    data = np.load(f'{path_data_eeg}/segment29.npz')
    savg_time = data['savg_time']
    savg_data = data['savg_data']
    teeg_time = data['eeg_time']
    teeg_data = data['eeg_data']
    
    # desired ouput 
    savg_y1y2 = savg_data[:, 1, :, 0]-savg_data[:, 2, :, 0]
    teeg_y1y2 = (teeg_data[:, 1, :, 0]-teeg_data[:, 2, :, 0])
    
    t1=1000
    t2=300000

    #source activity
    id = np.where((savg_time>=t1)*(savg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    source_data = savg_y1y2[idt1:idt2]

    #eeg
    id = np.where((teeg_time>=t1)*(teeg_time<=t2))
    idt1 = id[0][0]
    idt2 = id[0][-1]
    
    eeg_data = teeg_y1y2[idt1:idt2]
    
    print(np.array(source_data).shape)

    methods=['Pearson']#,'Spearman', 'Coherence', 'Phase-lock' ] #'Pearson', 'Coherence','Cross-cor'
    for meth in methods : 
        closest_mats,most_repeating_states, correlation_matrices, cluster_labels,distances, pca, reduced_data = DFC_visu_pipeline(eeg_data,meth,animated_visu=False, window_size=1000,overlap=500,tmax=t2/1000, intrahemispheric=False)
    closest_mats_joined.append(closest_mats)

In [ ]:

for i, subj in enumerate([2,5,7,8,9]):
    plt.figure()
    plt.imshow(closest_mats_joined[i][0][ordered_ind][ordered_ind],cmap='viridis')
    plt.title(f'Subject {subj}')
    plt.colorbar()
    plt.clim([0,1])
    



In [ ]:
closest_mats_joined[i][0][ordered_ind].shape

In [ ]:
ordered_closest_mats_joined = []
for i,closest_mats in enumerate(closest_mats_joined):
    ordered_closest_mats = []
    for matrix in closest_mats:
        ordered_matrix = matrix[ordered_ind[i]][:, ordered_ind[i]]
        ordered_closest_mats.append(ordered_matrix)
    ordered_closest_mats_joined.append(ordered_closest_mats)

for i, subj in enumerate([2,5,7,8,9]):
    for j in range(3):
        plt.figure()
        plt.imshow(ordered_closest_mats_joined[i][j],cmap='viridis')
        plt.title(f'Subject {subj}')
        plt.colorbar()


In [ ]:
closest_mats_joined = np.array(closest_mats_joined)
closest_mats_joined.shape
for i in closest_mats_joined:
    print(i.shape)
    for j in i: 
        print(j.shape) 
        print(j[ordered_ind][:, ordered_ind].shape)

In [ ]:
ordered_closest_mats_joined = np.array(ordered_closest_mats_joined)
ordered_closest_mats_joined.shape

In [ ]:
ordered_ind[i].shape